In [1]:
from finviz import get_news
import pandas as pd
# nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from os.path import exists
from pathlib import Path

pd.set_option('display.max_colwidth', 25)

<IPython.core.display.Javascript object>

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gdp/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
path1 = Path("data/sentiment/")

In [3]:
# stocks = ['AAPL', 'GOOG']

stocks = list(pd.read_pickle(("data/recommenders/2022/2022-01/2022-01-12/recommender_02_return_dataFrame.pkl"))['Symbol'])
len(stocks)

382

In [4]:
def sentiment_analysis(newS, stocks):
    for stock in stocks:
        dates = newS['Date']
        headlines = newS['News Headline']
        links = newS['Article Link']
        sources = newS['Source']

        parsed_news=[]
        for r in range(len(newS)):
            parsed_news.append([stock, dates[r], headlines[r], links[r], sources[r]])    

        # Sentiment Analysis
        analyzer = SentimentIntensityAnalyzer()
        columns = ["Symbol", "Date", "News Headline", "Article Link", "Source"]
        news = pd.DataFrame(parsed_news, columns=columns)
        scores = news["News Headline"].apply(analyzer.polarity_scores).tolist()
        df_scores = pd.DataFrame(scores)
        news = news.join(df_scores, rsuffix="_right")       

        # View Data
        news["Date"] = pd.to_datetime(news.Date).dt.date
        unique_ticker = news["Symbol"].unique().tolist()
        news_dict = {name: news.loc[news["Symbol"] == name] for name in unique_ticker}
        values = []     

    for stock in stocks:
        dataframe = news_dict[stock]
        dataframe = dataframe.set_index("Symbol")
        dataframe = dataframe.drop(columns=["News Headline"])
        mean = round(dataframe["compound"].mean() * 100, 0)
        values.append(mean)    

    df = pd.DataFrame(stocks, columns=["Symbol"])
    df["Mean Sentiment"] = values
    return df

In [11]:
def mini_news(stocks):
    for stock in stocks:
        print ('\nRecent News: ')
        print ('Getting data for ' + stock + '...\n')        
        if exists(path1 / f"df_single_news_{stock}.csv"):
            pass
        else:
            try:
                data_news = get_news(stock)
                df_single_news = pd.DataFrame((data_news), columns=['Date','News Headline','Article Link','Source'])
                df_single_news.to_csv(path1 / f"df_single_news_{stock}.csv")
                time.sleep(2.5)
            except:
                print(f'BAD TICKER {stock}')
                stocks.remove(stock)
    return stocks

In [14]:
def news_stand(stocks):
    
    for stock in stocks:
        df = sentiment_analysis([stock])


def run_sentiment(stocks):
    print(len(stocks))
    df = pd.DataFrame()
    symbols = []
    sentiments = []

    stocks = mini_news(stocks)
    print(len(stocks))

    for stock in stocks:
        if exists(path1 / f"{stock}_sentiment.csv"):
            fd = pd.read_csv(path1 / f"{stock}_sentiment.csv")
            symbols.append(fd['Symbol'].loc[0])
            sentiments.append(fd['Mean Sentiment'].loc[0])
            
        else:
            newS = pd.read_csv(path1 / f"df_single_news_{stock}.csv")
            fd = sentiment_analysis(newS, [stock])
            symbols.append(fd['Symbol'].loc[0])
            sentiments.append(fd['Mean Sentiment'].loc[0])
            fd.to_csv(path1 / f"{stock}_sentiment.csv")
            time.sleep(2.5)

    df['Symbol'] = symbols
    df['Mean Sentiment'] = sentiments
    return df

In [15]:
df_final = run_sentiment(stocks)

367
367

Recent News: 
Getting data for AFG...


Recent News: 
Getting data for ALB...


Recent News: 
Getting data for AMAT...


Recent News: 
Getting data for AMBA...


Recent News: 
Getting data for AMD...


Recent News: 
Getting data for AMG...


Recent News: 
Getting data for AMN...


Recent News: 
Getting data for AMRC...


Recent News: 
Getting data for AMTB...


Recent News: 
Getting data for AN...


Recent News: 
Getting data for ANAB...


Recent News: 
Getting data for ANET...


Recent News: 
Getting data for AOSL...


Recent News: 
Getting data for APRN...


Recent News: 
Getting data for APTS...


Recent News: 
Getting data for AQST...


Recent News: 
Getting data for AQUA...


Recent News: 
Getting data for ARCB...


Recent News: 
Getting data for ARQQ...


Recent News: 
Getting data for ASML...


Recent News: 
Getting data for ASPN...


Recent News: 
Getting data for ATEN...


Recent News: 
Getting data for ATKR...


Recent News: 
Getting data for ATSPT...


Recent News: 

In [16]:
df_final.to_pickle('data/recommenders/2022/2022-01/2022-01-12/recommender_03_return_dataFrame.pkl')

In [17]:
df_final

,Symbol,Mean Sentiment
0,AAWW,27.0
1,ACI,21.0
2,ACLS,19.0
3,AFG,21.0
4,ALB,23.0
...,...,...
362,XPEV,13.0
363,XPOF,42.0
364,YELL,12.0
365,ZIM,20.0
